# Data Preprocessing using RAPIDS and Training XGBoost for Fraud Detection

In this notebook we will walk through using RAPIDS for GPU-accelerated data preprocessing and training of XGBoost model for Fraud Detection use-case.

## Get Data

For this example, we use the Tabformer [synthetic credit card transactions dataset](https://arxiv.org/abs/1910.03033) from IBM available on [Kaggle](https://www.kaggle.com/datasets/ealtman2019/credit-card-transactions). You can either directly download the dataset from this [Kaggle link](https://www.kaggle.com/datasets/ealtman2019/credit-card-transactions) and then upload it to your SageMaker notebook instance. Or you may fetch the data from Kaggle command line client using the following commands.

**NOTE:** You will need to make sure that your Kaggle credentials are [available](https://github.com/Kaggle/kaggle-api#api-credentials) either through a kaggle.json file or via environment variables.

In [2]:
!pip install -q kaggle

^C
ERROR: Operation cancelled by user


In [3]:
!kaggle datasets download -d ealtman2019/credit-card-transactions

/bin/sh: kaggle: command not found


In [4]:
!unzip -u credit-card-transactions.zip

Archive:  credit-card-transactions.zip


## Data Preprocessing

In [1]:
import cudf
import cuml
from cuml.preprocessing import LabelEncoder
import numpy as np
import pickle
import os

In [2]:
data_path = './'

In [3]:
data_csv = 'credit_card_transactions-ibm_v2.csv'

In [4]:
data = cudf.read_csv(os.path.join(data_path, data_csv))

In [5]:
data.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,<NA>,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,<NA>,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,<NA>,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,<NA>,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,<NA>,No


In [6]:
data.shape

(24386900, 15)

In [7]:
data['Zip'] = data['Zip'].astype('object')
data['MCC'] = data['MCC'].astype('object')
data["Merchant Name"] = data["Merchant Name"].astype("object")

In [8]:
SEED = 42
data = data.sample(frac=0.5, random_state=SEED)
data = data.reset_index(drop=True)

In [9]:
data.shape

(12193450, 15)

### Encode labels


In [10]:
y = data['Is Fraud?']
data.drop(columns=['Is Fraud?'], inplace=True)
y = (y == "Yes").astype(int)

### Save subset for inference

We will save a subset of data to submit inference requests for later on in the second notebook.

In [11]:
data_infer = data.iloc[258:263]
data_infer.to_csv('data_infer.csv', index=False)

### Handle Missing Values

In [12]:
data.isna().sum()/len(data) * 100

User               0.000000
Card               0.000000
Year               0.000000
Month              0.000000
Day                0.000000
Time               0.000000
Amount             0.000000
Use Chip           0.000000
Merchant Name      0.000000
Merchant City      0.000000
Merchant State    11.159934
Zip               11.804764
MCC                0.000000
Errors?           98.408834
dtype: float64

In [13]:
data.loc[data["Merchant City"]=="ONLINE", "Merchant State"] = "ONLINE" 
data.loc[data["Merchant City"]=="ONLINE", "Zip"] = "ONLINE" 

In [14]:
data['Errors?'] = data['Errors?'].notna().astype(int)

In [15]:
us_states_plus_online = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', 'ONLINE']

data.loc[~data["Merchant State"].isin(us_states_plus_online), "Zip"] = "FOREIGN"

### Handle Amount and Time

In [16]:
data['Amount'] = data['Amount'].str.slice(1).astype('float32')
data['Hour'] = data['Time'].str.slice(stop=2).astype('int64')
data['Minute'] = data['Time'].str.slice(start=3).astype('int64')
data.drop(columns=['Time'], inplace=True)

###  Train Test Split

In [17]:
from cuml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=SEED, stratify=y)

In [18]:
# Free up some room on the GPU by explicitly deleting dataframes
import gc
del data
del y
gc.collect()

264

### Encoding Categorical Columns

In [19]:
categorial_columns = ['Zip', 'MCC', 'Merchant Name', 'Use Chip', 'Merchant City', 'Merchant State']
encoders = {}

# handle unknown values present in training data but not in test data
for col in categorial_columns:
    unique_values = X_train[col].unique().values_host
    X_test.loc[~X_test[col].isin(unique_values), col] = 'UNKNOWN'
    unique_values = np.append(unique_values, ['UNKNOWN'])
    # convert to cudf series
    unique_values = cudf.Series(unique_values)
    le = LabelEncoder().fit(unique_values)
    X_train[col] = le.transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
    encoders[col] = le.classes_.values_host

### Save Label Encoders to be used for preprocessing at Inference time

In [31]:
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f)

In [53]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

## Train XGBoost

In [21]:
import xgboost as xgb
import time

dtrain = xgb.DMatrix(
        X_train,
        y_train)

dtest = xgb.DMatrix(
        X_test,
        y_test)

max_depth = 8
num_trees = 2000
xgb_params = {
    'max_depth':          max_depth,
    'tree_method':       'gpu_hist',
    'objective':         'binary:logistic',
    'eval_metric':       'aucpr',
    'predictor':         'gpu_predictor',
}
start = time.time()
model = xgb.train(params=xgb_params, 
                       dtrain=dtrain, 
                       num_boost_round=num_trees)
print("Training Time", time.time()-start, "seconds")

Training Time 108.20628547668457 seconds


In [22]:
y_score = model.predict(dtest)
threshold = 0.5
y_pred = (y_score >= 0.5).astype(int)

In [23]:
from sklearn.metrics import f1_score

y_true = y_test.values_host
f1 = f1_score(y_true, y_pred)
print(f'Test F1-Score: {f1: 0.4f}')

Test F1-Score:  0.8502


### Save Trained Model

In [32]:
model_path = "./xgboost.json"

In [25]:
# model.save_model(model_path)

## CPU GPU FIL Benchmarks

**Note:** CPU Predictor can take ~2 minutes.

In [2]:
import xgboost as xgb
model_path = "./xgboost.json"
model = xgb.Booster()
model.load_model(model_path)

In [3]:
# %%time
# model.set_param({"predictor": "cpu_predictor"})
# cpu_preds = model.predict(dtest)

In [4]:
%%timeit
model.set_param({"predictor": "gpu_predictor"})
gpu_preds = model.inplace_predict(X_train)

1.81 s ± 303 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


###  Prediction using Forest Inference Library (FIL)

In [5]:
import cupy as cp
import cuml
model_path = "./xgboost.json"

In [6]:
# Call FIL inference from cuml and load model and params
fil = cuml.ForestInference.load(
      filename=model_path,                                    
      output_class=True,    
      model_type='xgboost_json')

In [7]:
%%timeit
fil_preds = fil.predict(X_train)

2.06 s ± 140 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
X_train_arr = cp.asarray(X_train.values, order='C')

In [9]:
%%timeit
fil_preds = fil.predict(X_train_arr)

1.96 s ± 176 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
